In [1]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [ ]:
#using Plots

In [ ]:
gmsh.open("wave.geo")

In [4]:
#gmsh.fltk.run()

In [ ]:
body = FEM.material("body", ν=0)
problem1 = FEM.Problem([body], type="PlaneStress", bandwidth="RCMK")
problem2 = FEM.Problem([body], type="PlaneHeatConduction")

In [ ]:
K1 = FEM.stiffnessMatrix(problem1)

In [ ]:
M1 = FEM.massMatrix(problem1, lumped=true)

In [ ]:
K2 = FEM.heatConductionMatrix(problem2)

In [ ]:
C2 = FEM.heatCapacityMatrix(problem2, lumped=false)

In [10]:
load = FEM.load("left", fx=0)
f1 = FEM.loadVector(problem1, [load]);

In [11]:
hf = FEM.heatFlux("left", qn=0)
q2 = FEM.heatFluxVector(problem2, [hf]);

In [12]:
right1 = FEM.displacementConstraint("right", ux=0);

In [13]:
u0 = FEM.initialDisplacement(problem1, "body", ux=0, uy=0)
v0 = FEM.initialVelocity(problem1, "body", vx=1000)
FEM.initialVelocity!(problem1, "right", v0, vx=0);

In [14]:
FEM.applyBoundaryConditions!(problem1, K1, M1, f1, [right1])

In [ ]:
Tₘᵢₙ = FEM.smallestPeriodTime(K1, M1)
Δt = Tₘᵢₙ / π

In [16]:
T0 = FEM.initialTemperature(problem2, "body", T=273);

In [17]:
using SparseArrays, LinearAlgebra
ϑ = 0.66
invM1 = spdiagm(1 ./ diag(M1))
nsteps = 500
dof1, dof1 = size(K1)
dof2, dof2 = size(K2)
u = zeros(dof1, nsteps)
v = zeros(dof1, nsteps)
T = zeros(dof2, nsteps)
t = zeros(nsteps)
a0 = M1 \ (f1 - K1 * u0)
u00 = u0 - v0 * Δt + a0 * Δt^2 / 2
u[:, 1] = u0
v[:, 1] = v0
T[:, 1] = T0
t[1] = 0
T00 = copy(T0)
u1 = u0
v1 = v0
T1 = T0
for i in 2:nsteps
    for j in 1:3
        LH2 = FEM.latentHeatMatrix(problem2, u1, v1, T00)
        fq1 = FEM.thermalLoadVector(problem1, T1, T₀=T00)
        u1 = 2.0 * u0 - u00 + Δt * Δt * invM1 * ((f1 + fq1) - K1 * u0)
        v1 = (u1 - u0) / Δt
        A = C2 + ϑ * Δt * (K2 - LH2)
        b = (C2 - (1 - ϑ) * Δt * (K2 - LH2)) * T0 + Δt * q2
        T1 = A \ b
    end

    u[:, i] = u1
    v[:, i] = v1
    T[:, i] = T1
    t[i] = t[i-1] + Δt
    u00 = u0
    u0 = u1
    T0 = T1
end

In [ ]:
T[:, size(T, 2)]

In [ ]:
TT = FEM.showDoFResults(problem2, T, t=t, "T")

In [20]:
e = FEM.solveStrain(problem1, u, DoFResults=true);

In [21]:
s = FEM.solveStress(problem1, u, DoFResults=true);

In [ ]:
ex = FEM.showDoFResults(problem1, e, t=t, "ex")

In [ ]:
sx = FEM.showDoFResults(problem1, s, t=t, "sx")

In [ ]:
TTp = FEM.plotOnPath(problem2, "path", TT, visible=true)

In [ ]:
expp = FEM.plotOnPath(problem1, "path", ex, visible=true)

In [ ]:
gmsh.fltk.run()

In [27]:
gmsh.finalize()